In [11]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [12]:
def Stem(input):
    x=layers.Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),padding='same',activation='relu')(input)
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
    x=layers.LayerNormalization()(x)
    x=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x=layers.Conv2D(filters=192,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x=layers.LayerNormalization()(x)
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)    
    return x


def Inception(x,f1,f2,f3,f4,f5,f6):

    x1=layers.Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
          
    x2=layers.Conv2D(filters=f2,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=f3,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x2)

    x3=layers.Conv2D(filters=f4 ,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x3=layers.Conv2D(filters=f5,kernel_size=(5,5),strides=(1,1),padding='same',activation='relu')(x3)
       
    x4=layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)
    x4=layers.Conv2D(filters=f6,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x4)
    
    return tf.concat([x1,x2,x3,x4],axis=3)

def AUX(input):
    ax=layers.AveragePooling2D(pool_size=(1,1),strides=(1,1))(input)
    ax=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(ax)
    ax=layers.Flatten()(ax)
    ax=layers.Dense(units=1024,activation='relu')(ax)
    ax=layers.Dropout(0.7)(ax)
    ax=layers.Dense(units=10,activation='softmax')(ax)
    
    return ax


def FClayer(input):
    fc=layers.AveragePooling2D(pool_size=(7,7),strides=(1,1))(input)
    fc=layers.Flatten()(fc)
    fc=layers.Dense(units=10,activation='softmax')(fc)    
    return fc

In [ ]:
inputs=Input(shape=(224,224,3))
x=Stem(inputs)
x=Inception(x,256,256,256,256,256,256)
x=Inception(x,480,480,480,480,480,480)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
x=Inception(x,512,512,512,512,512,512)

ax1=AUX(x)
x=Inception(x,512,512,512,512,512,512)
x=Inception(x,512,512,512,512,512,512)
x=Inception(x,528,528,528,528,528,528)
ax2=AUX(x)
x=Inception(x,832,832,832,832,832,832)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
x=Inception(x,832,832,832,832,832,832)
x=Inception(x,1024,1024,1024,1024,1024,1024)
output=FClayer(x)
   
GooogleNet = tf.keras.models.Model(inputs,[output,ax1,ax2],name = 'GoogleNet')
model=GooogleNet

In [ ]:
model.summary()

Model: "GoogleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                          

In [ ]:
plot_model(model,to_file='GoogleNet.png',show_shapes=True)

In [ ]:
import os
import cv2
label_list=['ADONIS','AFRICAN GIANT SWALLOWTAIL','AMERICAN SNOOT','AN 88','APPOLLO','ARCIGERA FLOWER MOTH','ATALA',
'ATLAS MOTH','BANDED ORANGE HELICONIAN','BANDED PEACOCK']
img_list=[]

for i in range(len(label_list)):
    dir=os.listdir('D:\\dataset\\butterfly\\train\\{0}/'.format(label_list[i]))


    for j in range(len(dir)):
        path='D:\\dataset\\butterfly\\train\\{0}/'.format(label_list[i]) + dir[j]
        image=cv2.imread(path,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        arr=np.array(image)
        img_list.append(arr)

    img=np.array(img_list)

# 이미지 변환행렬 shape 확인
img.shape

(1257, 224, 224, 3)

In [ ]:
label_num=[125, 107, 105, 121, 128, 140, 143, 129, 139, 119]

lst=np.array([[1,0,0,0,0,0,0,0,0,0]])


for i in range(10):
    lst_temp=np.array([[0,0,0,0,0,0,0,0,0,0]])
    lst_temp[0,i]=1
    for j in range(label_num[i]):
        lst=np.vstack((lst,lst_temp))

lst.shape

(1257, 10)

In [ ]:
train_x,val_x,train_y,val_y = train_test_split(img, lst,test_size=0.1, random_state=10)

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['acc'],loss_weights={'dense_4':1, 'dense_1':0., 'dense_3':0})

In [ ]:
#'dense_4', 'dense_1', 'dense_3'

history=model.fit(train_x,{'dense_4':train_y,'dense_1':train_y,'dense_3':train_y},validation_data=(test_x,test_y),epochs=10,batch_size=8,verbose=1)

In [18]:
#'dense_4', 'dense_1', 'dense_3'

history=model.fit(img,{'dense_4':lst,'dense_1':lst,'dense_3':lst},epochs=10,batch_size=8,verbose=1)

Epoch 1/10
158/158 [==============================] - 1053s 7s/step - loss: 2.3819 - dense_4_loss: 2.3819 - dense_1_loss: 2.3881 - dense_3_loss: 2.3028 - dense_4_acc: 0.0963 - dense_1_acc: 0.1034 - dense_3_acc: 0.1058
Epoch 2/10
158/158 [==============================] - 1076s 7s/step - loss: 2.3004 - dense_4_loss: 2.3004 - dense_1_loss: 2.3026 - dense_3_loss: 2.2995 - dense_4_acc: 0.1074 - dense_1_acc: 0.1249 - dense_3_acc: 0.1042
Epoch 3/10
158/158 [==============================] - 1085s 7s/step - loss: 2.3000 - dense_4_loss: 2.3000 - dense_1_loss: 2.3065 - dense_3_loss: 2.2997 - dense_4_acc: 0.1114 - dense_1_acc: 0.1169 - dense_3_acc: 0.1090
Epoch 4/10
103/158 [==================>...........] - ETA: 6:19 - loss: 2.2995 - dense_4_loss: 2.2995 - dense_1_loss: 2.3000 - dense_3_loss: 2.3001 - dense_4_acc: 0.1007 - dense_1_acc: 0.1080 - dense_3_acc: 0.0983

KeyboardInterrupt: 